In [1]:
environment_library = 'gym'  # @param ['dm_control', 'gym']

In [7]:
import IPython

from acme import environment_loop
from acme import specs
from acme import wrappers
# from acme.agents.tf import d4pg
# from acme.tf import networks
# from acme.tf import utils as tf2_utils
from acme.utils import loggers 
import numpy as np
import sonnet as snt
import time
from acme.agents.jax import d4pg
from acme.jax import networks
from acme.jax import utils as jax_utils
import haiku as hk
from acme.agents.jax.d4pg import networks as d4pg_network
from acme.agents.jax.d4pg import config as d4pg_config

# Import the selected environment lib
if environment_library == 'dm_control':
    from dm_control import suite
elif environment_library == 'gym':
    import gym

# Imports required for visualization
import pyvirtualdisplay
import imageio
import base64

# Set up a virtual display for rendering.
display = pyvirtualdisplay.Display(visible=0, size=(1400, 900)).start()



In [4]:
environment = gym.make('Pendulum-v1')
environment = wrappers.GymWrapper(environment)  # To dm_env interface.


# Make sure the environment outputs single-precision floats.
environment = wrappers.SinglePrecisionWrapper(environment)

# Grab the spec of the environment.
environment_spec = specs.make_environment_spec(environment)

In [39]:
# #@title Build agent networks

# # Get total number of action dimensions from action spec.
# num_dimensions = np.prod(environment_spec.actions.shape, dtype=int)

# # Create the shared observation network; here simply a state-less operation.
# observation_network = jax_utils.batch_concat

# def policy_network():
#     policy_network = hk.Sequential([
#     networks.LayerNormMLP((256, 256, 256), activate_final=True),
#     networks.NearZeroInitializedLinear(num_dimensions),
#     networks.TanhToSpec(environment_spec.actions),
# ])
#     return policy_network
    
# policy_network = hk.transform(policy_network)

# def critic_network():
#     critic_network = snt.Sequential([
#     # The multiplexer concatenates the observations/actions.
#     networks.CriticMultiplexer(),
#     networks.LayerNormMLP((512, 512, 256), activate_final=True),
#     networks.DiscreteValuedHead(vmin=-150., vmax=150., num_atoms=51),
# ])
#     return critic_network
    
    
# critic_network = hk.transform(critic_network)



In [5]:

net = d4pg_network.make_networks(environment_spec)



# Create a logger for the agent and environment loop.
agent_logger = loggers.TerminalLogger(label='agent', time_delta=10.)
env_loop_logger = loggers.TerminalLogger(label='env_loop', time_delta=10.)

# Create the D4PG agent.
agent = d4pg.D4PG(
    spec=environment_spec,
    network = net,
    config = d4pg_config.D4PGConfig,
    random_seed = 1
)

# Create an loop connecting this agent to the environment created above.
env_loop = environment_loop.EnvironmentLoop(
    environment, agent, logger=env_loop_logger)



[reverb/cc/platform/tfrecord_checkpointer.cc:150]  Initializing TFRecordCheckpointer in /tmp/tmpasspo7hn.
[reverb/cc/platform/tfrecord_checkpointer.cc:386] Loading latest checkpoint from /tmp/tmpasspo7hn
[reverb/cc/platform/default/server.cc:71] Started replay server on port 23167


In [8]:


tic = time.perf_counter()
env_loop.run(num_episodes=1000)
toc = time.perf_counter()

print ("time elapsed (s): {toc - tic:0.4f}")


[reverb/cc/client.cc:165] Sampler and server are owned by the same process (24657) so Table priority_table is accessed directly without gRPC.
[reverb/cc/client.cc:165] Sampler and server are owned by the same process (24657) so Table priority_table is accessed directly without gRPC.
[reverb/cc/client.cc:165] Sampler and server are owned by the same process (24657) so Table priority_table is accessed directly without gRPC.
[reverb/cc/client.cc:165] Sampler and server are owned by the same process (24657) so Table priority_table is accessed directly without gRPC.


time elapsed (s): {toc - tic:0.4f}


In [10]:
print ("time elapsed (s): {}".format(toc - tic))

time elapsed (s): 130.53202583599887


In [11]:


# Create a simple helper function to render a frame from the current state of
# the environment.
if environment_library == 'dm_control':
    def render(env):
        return env.physics.render(camera_id=0)
elif environment_library == 'gym':
    def render(env):
        return env.environment.render(mode='rgb_array')
else:
    raise ValueError(
      "Unknown environment library: {};".format(environment_library) +
      "choose among ['dm_control', 'gym'].")

def display_video(frames, filename='temp.mp4'):
    """Save and display video."""

  # Write video
    with imageio.get_writer(filename, fps=60) as video:
        for frame in frames:
            video.append_data(frame)

  # Read video and display the video
    video = open(filename, 'rb').read()
    b64_video = base64.b64encode(video)
    video_tag = ('<video  width="320" height="240" controls alt="test" '
               'src="data:video/mp4;base64,{0}">').format(b64_video.decode())

    return IPython.display.HTML(video_tag)



In [12]:


timestep = environment.reset()
frames = [render(environment)]

while not timestep.last():
  # Simple environment loop.
    action = agent.select_action(timestep.observation)
    timestep = environment.step(action)

    # Render the scene and add it to the frame stack.
    frames.append(render(environment))

# Save and display a video of the behaviour.
display_video(np.array(frames))



[swscaler @ 0x588d680] Warning: data is not aligned! This can lead to a speed loss
